### XML 형태의 Oxford Pets 데이터 세트를 이용하여 Object Detection 및 Evaluation

### 본 실습 예제는 GPU를 활용하므로 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택해 주십시요.

### tensorflow, keras 설치 및 강의 실습코드/데이터 Download

Colab 버전은 colab에서 pip 명령어를 이용하여 tensorflow 1.13, keras 2.2를 소스코드 커널 기동시 마다 설치해야 합니다

Colab의 tensorflow는 2020년 기준으로 2.2 이며, keras는 2.3입니다. 실습코드는 tensorflow 1.13과 1.15, keras 2.2 기준으로 되어 있으므로 이를 downgrade해야 합니다.

pip를 이용하여 tensorflow 1.13을 설치하면 자동으로 downgrade 됩니다.

OpenCV는 Colab에서 이미 설치 되어 있으니 추가설치는 필요 없습니다.

강의 실습코드와 데이터는 https://github.com/chulminkw/DLCV.git 에서 다운로드 할 수 있습니다. 

In [ ]:
# 현재 디렉토리는 /content이며 이 디렉토리를 기준으로 실습코드와 데이터를 다운로드 합니다. 
!pwd
!rm -rf DLCV
!git clone https://github.com/chulminkw/DLCV.git
# DLCV 디렉토리가 Download되고 DLCV 밑에 Detection과 Segmentation 디렉토리가 있는 것을 확인
!ls -lia 
!ls -lia DLCV

# tensorflow 1.15을 설치합니다. 자동으로 tensorflow 2.2가 1.15으로 downgrade 됩니다. 
!pip install tensorflow-gpu==1.15.2 
# keras 2.3를 설치합니다. 
!pip install keras==2.3.0

#### tensorflow, keras 버전 및 GPU가 세팅되어 있는지 확인. 

In [ ]:
# tensorflow는 1.15, keras는 2.3 버전 확인
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

# gpu가 세팅되어 있는지 확인. 
tf.test.gpu_device_name()

In [ ]:
# fizyr keras-retinanet 다운로드 
%cd /content/DLCV/Detection/retina
!rm -rf /content/DLCV/Detection/retina/keras-retinanet
!git clone https://github.com/fizyr/keras-retinanet.git

# fizyr keras-retinanet 설치
%cd /content/DLCV/Detection/retina/keras-retinanet
!pip install . --user
!python setup.py build_ext --inplace

In [ ]:
# 아래 import로 keras-retinanet이 정상적으로 설치되어 있는지 확인. 특히 backbone 함수 체크. 
import tensorflow as tf

from keras_retinanet import models
from keras_retinanet.models import backbone

b = backbone('resnet50')

#### Oxford pet 데이터 세트 다운로드
* oxford pet 데이터 세트를 다운로드 한 뒤 압축을 풀고, 불필요한 파일 삭제
* 기존 annotation과 image 처리 로직과 유사하게 annotations 디렉토리에 xml 파일을 옮기고, images 디렉토리에 모든 image 파일 정리. 

In [ ]:
# oxford pet 데이터 세트 다운로드 
%cd /content/DLCV/data/
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
# ox_pet 디렉토리를 /content/DLCV/data 밑에 생성한 후 다운로드 된 압축파일을 이동 시킴. 
!rm -rf ox_pet
!mkdir ox_pet
!mv images.tar.gz ox_pet
!mv annotations.tar.gz ox_pet
# 압축 파일 해제 
%cd /content/DLCV/data/ox_pet
!tar -xvf images.tar.gz > /dev/null 2>&1
!tar -xvf annotations.tar.gz > /dev/null 2>&1
# 불필요한 디렉토리를 삭제하고 annotations/xmls 디렉토리에 있는 모든 xml 파일들을 annotations 디렉토리로 이동한 뒤 xmls 디렉토리 삭제 
%cd /content/DLCV/data/ox_pet/annotations
!rm -rf list.txt README test.txt trainval.txt trimaps
%cd /content/DLCV/data/ox_pet/annotations/xmls
!mv *.xml ..
%cd  /content/DLCV/data/ox_pet/annotations
!rm -rf xmls

# annotation 파일과 이미지 파일의 갯수 확인. 
!ls -lia /content/DLCV/data/ox_pet/images |wc -l
!ls /content/DLCV/data/ox_pet//annotations |wc -l

#### annotation 디렉토리의 파일 확인

In [ ]:
# annotation과 image 디렉토리 설정. annotation디렉토리에 있는 파일 확인. 
import os
from pathlib import Path

# 코랩 버전 수정
#HOME_DIR = str(Path.home())
HOME_DIR = '/content'
ANNO_DIR = os.path.join(HOME_DIR, 'DLCV/data/ox_pet/annotations')
IMAGE_DIR = os.path.join(HOME_DIR, 'DLCV/data/ox_pet/images')
print(ANNO_DIR)
print('IMAGE 파일 개수는:',len(os.listdir(IMAGE_DIR)), 'XML 파일 개수:', len(os.listdir(ANNO_DIR)))
os.listdir(ANNO_DIR)

In [ ]:
!cat /content/DLCV/data/ox_pet/annotations/staffordshire_bull_terrier_128.xml

In [ ]:
# 전체 파일에서 고유한 품종을 확인. 
files = os.listdir(ANNO_DIR)
file_breed = [file[0:file.rfind('_')] for file in files if 'xml' in file]
breed = list(set(file_breed))

print(len(breed))
print(breed)

#### XML 파일을 읽어 CSV 형태의 파일로 생성하고 이를 pet_anno.csv 파일로 저장

In [ ]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET

## filename에서 class명을 가져옴. xml 파일에는 class명이 class 대분류값인 cat/dog으로 되어 있음. 
def get_class_name_from_filename(file_name):
    file_breed = file_name[0:file_name.rfind('_')]
    return file_breed

# XML 파일을 Pandas DataFrame으로 변환 한뒤 DataFrame의 to_csv()를 이용하여 csv 파일로 생성하고 DataFrame반환
def xml_to_csv(path, output_filename):
    xml_list = []
    # xml 확장자를 가진 모든 파일의 절대 경로로 xml_file할당. 
    for xml_file in glob.glob(path + '/*.xml'):
        # xml 파일을 parsing하여 XML Element형태의 Element Tree를 생성하여 object 정보를 추출. 
        tree = ET.parse(xml_file)
        root = tree.getroot()
        # 파일내에 있는 모든 object Element를 찾음. 
        for obj in root.findall('object'):
            # filename, 이미지파일 크기, class명은 get_clas_name_from_filename()함수로 생성, 그리고 bounding box 위치 추출.
            value = (os.path.join(IMAGE_DIR, root.find('filename').text),
                    int(obj[4][0].text),
                    int(obj[4][1].text),
                    int(obj[4][2].text),
                    int(obj[4][3].text),
                    get_class_name_from_filename(root.find('filename').text),
                    )
            # object별 정보를 tuple형태로 xml_list에 저장. 
            xml_list.append(value)
    # 모든 object별 정보를 DataFrame으로 생성하고 이를 CSV 파일로 생성하고 DataFrame은 반환. 
    column_name = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class_name']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    xml_df.to_csv(os.path.join(path,output_filename), index=None, header=None)
    return xml_df

In [ ]:
# annotation 디렉토리 밑에 pet_anno.csv로 저장
pet_df = xml_to_csv(ANNO_DIR, os.path.join(ANNO_DIR, 'pet_anno.csv'))

In [ ]:
os.path.join(ANNO_DIR, 'pet_anno.csv')

In [ ]:
!sort /content/DLCV/data/ox_pet/annotations/pet_anno.csv

#### class명과 class id 명 매핑을 클래스명의 알파벳 순으로 0부터 차례로 임의 매핑하고 이를 pet_class.txt파일에 저장. 

In [ ]:
class_names = pet_df.groupby('class_name')['class_name'].max().to_list()
class_ids = list(range(0, len(class_names)))
print(class_names, class_ids)
pd.DataFrame({'class_name':class_names, 'class_id':class_ids}).to_csv(os.path.join(ANNO_DIR, 'pet_class.txt'), header=None, index=None)

In [ ]:
pet_df.groupby('class_name')['class_name'].max()

In [ ]:
!cat /content/DLCV/data/ox_pet/annotations/pet_class.txt

In [ ]:
class_names = pet_df.groupby('class_name')['class_name'].max().to_list()
class_ids = list(range(0, len(class_names)))
labels_to_names = pd.DataFrame({'class_name':class_names, 'class_Id':class_ids}).to_dict()['class_name']
labels_to_names

#### Oxford pets 데이터 세트 학습

#### keras-retinanet으로 pretrained된 coco 모델 다운로드하고 해당 모델을 로드
* 앞 예제에서 pretrained 모델을 생성했지만 코렙 버전은 재 생성해야 함. 
* 코렙 버전은 /content/DLCV/Detection/retina/keras-retinanet/snapshots 디렉토리 밑에 pretrained 모델을 download

In [ ]:
# 아래 모델은 https://github.com/fizyr/keras-retinanet/releases 에서 download 받을 수 있음. 
# 해당 모델 h5 파일을 snapshot 디렉토리에 저장 후 retina model의 load_model()을 이용하여 모델 로딩.
%cd  /content/DLCV/Detection/retina/keras-retinanet/snapshots
!wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5 

In [ ]:
import numpy as np
import cv2
from os import listdir, walk
import math
import tensorflow as tf
from os.path import join
from keras_retinanet.bin.train import create_generators,create_models,create_callbacks
from keras_retinanet.models import backbone,load_model,convert_model
from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
from keras_retinanet.utils.visualization import draw_boxes

#from imgaug import augmenters as iaa

tf.set_random_seed(31) # SEEDS MAKE RESULTS MORE REPRODUCABLE
np.random.seed(17)

In [ ]:
b = backbone('resnet50')
files = os.listdir(ANNO_DIR)


class args:
    batch_size = 16
    config = None
    random_transform = True # Image augmentation
    annotations = os.path.join(ANNO_DIR, 'pet_anno.csv')
    val_annotations = None
    classes = os.path.join(ANNO_DIR, 'pet_class.txt')
    image_min_side = 800
    image_max_side = 1333
    no_resize=None
    dataset_type = 'csv'
    tensorboard_dir = ''
    evaluation = False
    snapshots = True
    snapshot_path = '/content/DLCV/Detection/retina/keras-retinanet/snapshots/ox_pet'
    backbone = 'resnet50'
    epochs = 50
    steps = len(files)//(batch_size)
    weighted_average = True
    # 코랩 버전 신규 추가
    reduce_lr_factor=0.5
    reduce_lr_patience=5

In [ ]:
# train용 generator 생성, valid용 generator는 데이터 부족으로 위 args 설정에서 None으로 함. 
train_gen,valid_gen = create_generators(args,b.preprocess_image)

# retinanet 기반 네트웍 모델 설정. weight값을 아직 설정하지 않았으며, args config 설정. 
# model, training_model, prediction_model이 반환되나 이중 training_model만 사용
model, training_model, prediction_model = create_models(
            backbone_retinanet=b.retinanet,
            num_classes=train_gen.num_classes(),
            weights=None,
            multi_gpu=False,
            freeze_backbone=True,
            lr=1e-3,
            config=args.config)

# callback 생성. epoch시 마다 발생하는 ModelCheckpoint, ReduceLROnPlateur callback 설정. 
callbacks = create_callbacks(model, training_model, prediction_model, valid_gen,args)

In [ ]:
# 학습 모델의 초기 가중치를 coco pretrained weight로 설정
training_model.load_weights('/content/DLCV/Detection/retina/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',skip_mismatch=True,by_name=True)

# 학습 수행. 코랩 버전에서는 학습에 매우 많은 시간이 소모됨. 모델 정확도가 떨어지더라도 epoch횟수를 10이하로 변경 필요
# 코랩 버전에서 학습 수행 시간이 너무 오래 걸릴 경우 이미 학습된 모델을 다운로드 받아서 inference 수행 고려 
training_model.fit_generator(train_gen, steps_per_epoch=args.steps, epochs=args.epochs, verbose=1, callbacks=callbacks)

In [ ]:
# 코랩 버전에서 학습 수행 시간이 너무 오래 걸릴 경우 이미 학습된 모델을 다운로드 받아서 inference 수행
# 학습된 모델은 github에서 다운로드 가능: https://github.com/chulminkw/DLCV/releases/download/1.0/resnet50_csv_50.h5
# 아래 주석을 해제하고 github에서 해당 모델 download 적용 고려. 
%cd /content/DLCV/Detection/retina/keras-retinanet/snapshots
!mkdir ox_pet
%cd /content/DLCV/Detection/retina/keras-retinanet/snapshots/ox_pet
!wget https://github.com/chulminkw/DLCV/releases/download/1.0/resnet50_csv_50.h5
!ls


#### 학습 모델을 inference 모델로 변경

In [ ]:
!cd ./keras-retinanet/snapshots; ls -lia
!/content/DLCV/Detection/retina/keras-retinanet/keras_retinanet/bin/convert_model.py /content/DLCV/Detection/retina/keras-retinanet/snapshots/ox_pet/resnet50_csv_50.h5 /content/DLCV/Detection/retina/keras-retinanet/snapshots/ox_pet/pet_inference.h5

In [ ]:
from keras_retinanet import models

# 코랩 버전 절대 경로 변경. 
model_path = os.path.join('/content/DLCV/Detection/retina/keras-retinanet','snapshots/ox_pet/pet_inference.h5')
print(model_path)
# load retinanet model
pet_retina_model = models.load_model(model_path, backbone_name='resnet50')

#### class id와 class name 매핑

In [ ]:
class_names = pet_df.groupby('class_name')['class_name'].max().to_list()
class_ids = list(range(0, len(class_names)))
labels_to_names = pd.DataFrame({'class_name':class_names, 'class_id':class_ids}).to_dict()['class_name']
labels_to_names

#### inference 모델을 이용하여 이미지 Object Detection

In [ ]:
import cv2
import numpy as np
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

def get_detected_image_retina(model, img_array, use_copied_array, is_print=True):
    
    # copy to draw on
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    img_array = preprocess_image(img_array)
    img_array, scale = resize_image(img_array)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img_array, axis=0))
    if is_print:
        print("object detection 처리 시간: ", round(time.time() - start,5))
    
    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw_img, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw_img, b, caption)
    
    if is_print:
        print("이미지 processing 시간: ", round(time.time() - start,5))
    
    return draw_img

In [ ]:
# os.listdir(IMAGE_DIR)

In [ ]:
import time 
import matplotlib.pyplot as plt
%matplotlib inline

# 'Sphynx_24.jpg' 'Russian_Blue_212.jpg', 'american_bulldog_66.jpg', 'pug_183.jpg'
img_array  = cv2.imread(os.path.join(IMAGE_DIR, 'Russian_Blue_212.jpg'))
detected_image = get_detected_image_retina(pet_retina_model,img_array, use_copied_array=True, is_print=True)

plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(detected_image)
plt.show()

#### 임의의 파일들을 Object Detection시각화 

In [ ]:
import numpy as np
from PIL import Image
np.random.seed(120)

# 모든 이미지 파일중에서 임의의 16개 파일만 설정. 
all_image_files = glob.glob(IMAGE_DIR + '/*.jpg')
all_image_files = np.array(all_image_files)
file_cnt = all_image_files.shape[0]
show_cnt = 16

show_indexes = np.random.choice(file_cnt, show_cnt)
show_files = all_image_files[show_indexes]
print(show_files)

detected_images = []
for filename in show_files:
    img_array = cv2.imread(os.path.join(IMAGE_DIR, filename))
    detected_image = get_detected_image_retina(pet_retina_model,img_array, use_copied_array=True, is_print=True)
    img_rgb = cv2.cvtColor(detected_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(detected_image)
    plt.show()

#### 학습된 모델의 Object Detection 성능 평가 

In [ ]:
import os
from pathlib import Path
# 코랩 버전 수정
#HOME_DIR = str(Path.home())
HOME_DIR = '/content'
ANNO_DIR = os.path.join(HOME_DIR, 'DLCV/data/ox_pet/annotations')

class args:
    batch_size=16
    dataset_type='csv'
    score_threshold=0.05
    iou_threshold=0.5
    max_detections=100
    image_min_side=800
    image_max_side=1333
    config=None
    annotations=os.path.join(ANNO_DIR, 'pet_anno.csv')
    classes=os.path.join(ANNO_DIR, 'pet_class.txt')
    #keras retinanet 패키지의 내부 버전 변경으로 아래가 추가 되어야 합니다.
    backbone='resnet50'

In [ ]:
from keras_retinanet.bin.evaluate import create_generator as eval_create_generator
# keras retinanet 패키지의 내부 버전 변경으로 기존 generator = eval_create_generator(args)는 삭제되고 아래로 변경되어야 합니다.
#generator = eval_create_generator(args)
backbone = models.backbone(args.backbone)
generator = eval_create_generator(args, backbone.preprocess_image)

In [ ]:
# 아래 코드는 코랩에서 22분동안 수행됩니다. 
from keras_retinanet.utils.eval import evaluate

average_precisions, inference_time = evaluate(
            generator,
            pet_retina_model,
            iou_threshold=args.iou_threshold,
            score_threshold=args.score_threshold,
            max_detections=args.max_detections,
            save_path=None
        )

In [ ]:
# print evaluation
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
          generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)

if sum(total_instances) == 0:
    print('No test instances found.')

print('Inference time for {:.0f} images: {:.4f}'.format(generator.size(), inference_time))

print('mAP using the weighted average of precisions among classes: {:.4f}'.format(sum([a * b for a, b in zip(total_instances, precisions)]) / sum(total_instances)))
print('mAP: {:.4f}'.format(sum(precisions) / sum(x > 0 for x in total_instances)))